In [8]:
import numpy as np
from scipy import ndimage
from scipy import signal
from scipy.io import loadmat
# import matplotlib.pyplot as plt
# %matplotlib inline

In [14]:
def expSnR(filenames, dt, StiType, channelnumbers):
    rstate_list = []
    x_list = []
    T_list = []
    if StiType == 'LED':
        for filename in filenames:
            annots = loadmat(filename)
            sampling_rate = 20000
            TimeStamps =np.round(np.squeeze(annots['TimeStamps']))
            TimeStamps = TimeStamps.astype(int)
            x = annots['a_data'][0, TimeStamps[0]*sampling_rate:TimeStamps[1]*sampling_rate+1]
            x = ndimage.gaussian_filter1d(x, sigma=int(sampling_rate*dt/5), mode='reflect') / dt
            x = x[::int(sampling_rate*dt)]
            x = x.astype(float)
            x = (x -np.mean(x))/np.std(x)
            T=np.arange(len(x))*dt+dt
            rstate = np.zeros((60, len(x)))
            for cn in channelnumbers:
                rstate[cn-1,:], _ = np.histogram(annots['Spikes'][0][cn-1]-TimeStamps[0], np.append(0,T))
            rstate_list.append(rstate.copy())
            x_list.append( x.copy())
            T_list.append( T.copy())
    elif StiType == 'OLED':
        for filename in filenames:
            annots = loadmat(filename)
            x = np.squeeze(annots['bin_pos'])
            T=np.arange(len(x))*dt+dt
            rstate = np.zeros((60, len(x)))
            for cn in channelnumbers:
                rstate[cn-1,:], _ = np.histogram(annots['reconstruct_spikes'][0][cn-1], np.append(0,T))
            rstate_list.append(rstate.copy())
            x_list.append( x.copy())
            T_list.append( T.copy())
    return x_list, T_list, rstate_list

In [10]:
def SSG(Fc_list, dt, tot, tau = 1, D = 1):
    T = np.arange(0,tot,dt)
    # OU ; input
    OU=np.zeros(len(T))
    mu, sigma = 0, 1
    for i in range(len(T)-1):
        OU[i+1]=OU[i]*(1-dt/tau)+np.sqrt(D*dt)*np.random.normal(mu,sigma)
    # filtering for OUSmoothed
    
    LPOU_list = []
    for Fc in Fc_list:
        b, a = signal.butter(2, 2*Fc*dt, btype='low', analog=False)
        LPOU_list.append(signal.filtfilt(b, a, OU))
    LPOU_list.append(OU)
    return LPOU_list